In [1]:
import h5py
import caffe
import numpy as np

In [7]:
maxWords = 100
# dataPath = '/data/gengshan/pose_s2vt/hdf5/buffer_32_s2vt_100/small_train_batches/batch_0.h5'
dataPath = '/data/gengshan/pose_s2vt/hdf5/buffer_32_s2vt_100/train_batches/batch_0.h5'
LSTM_NET_FILE = './captioner.prototxt'
# MODEL_FILE = '/data/gengshan/pose_s2vt/snapshots/stored/s2vt_asl_pose_iter_11668.caffemodel'
MODEL_FILE = '/data/gengshan/pose_s2vt/snapshots/s2vt_asl_pose_iter_40809.caffemodel'
caffe.set_mode_gpu()
caffe.set_device(0)
lstm_net = caffe.Net(LSTM_NET_FILE, MODEL_FILE, caffe.TEST)
vocabFile = '/data/gengshan/s2vt/vocabulary_whole.txt'
UNK_IDENTIFIER = '<en_unk>'

In [8]:
def vocab_inds_to_sentence(vocab, inds):
    sentence = ' '.join([vocab[i] for i in inds])
    # Capitalize first character.
    sentence = sentence[0].upper() + sentence[1:]
    # Replace <EOS> with '.', or append '...'.
    if sentence.endswith(' ' + vocab[0]):
        sentence = sentence[:-(len(vocab[0]) + 1)] + '.'
    else:
        sentence += '...'
    return sentence

def init_vocab_from_file(vocabFilePath):
    # initialize the vocabulary with the UNK word
    vocabulary = {UNK_IDENTIFIER: 0}
    vocabulary_inverted = [UNK_IDENTIFIER]
    num_words_dataset = 0
    with open(vocabFilePath, 'r') as f:
        for line in f.readlines():
            split_line = line.split()
            word = split_line[0]
            # print word
            if unicode(word) == UNK_IDENTIFIER:
                continue
            else:
                assert word not in vocabulary
            num_words_dataset += 1
            vocabulary[word] = len(vocabulary_inverted)
            vocabulary_inverted.append(word)
    num_words_vocab = len(vocabulary.keys())
    print ('Initialized vocabulary from file with %d unique words ' +
       '(from %d total words in dataset).') % \
      (num_words_vocab, num_words_dataset)
    assert len(vocabulary_inverted) == num_words_vocab
    return vocabulary_inverted

In [9]:
vocabList =  ['<EOS>'] + init_vocab_from_file(vocabFile)

wholeData = h5py.File(dataPath, 'r')
poseData = []
targetSent = []
existIdict = []
stageIdict = []
inputSent = []

streamSize,batchSize,_ = wholeData['frame_fc7'].shape
for streamIdx in range(0, streamSize/maxWords):
    for batchIdx in range(0, batchSize):
        poseData.append(wholeData['frame_fc7'][streamIdx * maxWords: (streamIdx + 1) * maxWords, batchIdx: batchIdx + 1, :])
        inputSent.append(wholeData['input_sentence'][streamIdx * maxWords: (streamIdx + 1) * maxWords, batchIdx: batchIdx + 1])
        existIdict.append(wholeData['cont_sentence'][streamIdx * maxWords: (streamIdx + 1) * maxWords, batchIdx: batchIdx + 1])
        stageIdict.append(wholeData['stage_indicator'][streamIdx * maxWords: (streamIdx + 1) * maxWords, batchIdx: batchIdx + 1])
        targetSent.append(wholeData['target_sentence'][streamIdx * maxWords: (streamIdx + 1) * maxWords, batchIdx: batchIdx + 1])

Initialized vocabulary from file with 6067 unique words (from 6066 total words in dataset).


In [10]:
for it in range(0, len(inputSent)):
    probs = lstm_net.forward(frames_fc7=poseData[it], cont_sentence=existIdict[it], input_sentence=inputSent[it],\
                             stage_indicator=stageIdict[it])['probs']
    
    predictIdx = np.squeeze(stageIdict[it] * np.argmax(probs, axis=2))

    predictIdx = predictIdx[np.where(stageIdict[it] != 0)[0][0]:]
    if 0 in predictIdx:
        predictIdx = predictIdx[:np.where(predictIdx == 0)[0][0]]
    
    print('\n' + str(it))
    print(vocab_inds_to_sentence(vocabList, [int(x) for x in predictIdx]))
    print(vocab_inds_to_sentence(vocabList, [int(x) for x in targetSent[it] if x != -1]))


0
<en_unk> <en_unk> <en_unk> <en_unk>...
<en_unk> <en_unk> and <en_unk> a runaway <en_unk> runaway from.

1
<en_unk> <en_unk> and <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk>...
Were <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> and <en_unk> <en_unk> <en_unk>.

2
<en_unk> <en_unk> <en_unk>...
<en_unk> tough <en_unk> <en_unk> putting me on the <en_unk>.

3
<en_unk> <en_unk> the <en_unk> of to <en_unk> us to <en_unk>...
<en_unk> of <en_unk> <en_unk> soon time for us to <en_unk>.

4
<en_unk> <en_unk> <en_unk>...
<en_unk> and welcome.

5
<en_unk> the <en_unk> opportunity in the <en_unk> of <en_unk> is diversity...
Editing a special section in the journal of child developed call.

6
<en_unk> <en_unk> <en_unk> we <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk> <en_unk>...
<en_unk> medical <en_unk>.

7
<en_unk> <en_unk> <en_unk